#### **Gruppo 5.b**: Caponi Marco (matricola: 508773) - Ceneda Gianluca (matricola: 488257)

# ANALISI E REVISIONE DEL PROGETTO LARSPLITTING 2D 

## CLASSE REFACTORING: boxcovering

Variabili utili per testare il funzionamento


In [17]:
using LinearAlgebraicRepresentation
Lar = LinearAlgebraicRepresentation
using IntervalTrees
using SparseArrays
using NearestNeighbors
using BenchmarkTools
using OrderedCollections
using Base.Threads


In [18]:
V = hcat([[0.,0],[1,0],[1,1],[0,1],[2,1]]...);    #vertici del modello 2D
V3 = hcat([[0.,0,0],[1,0,3],[1,1,2],[0,1,1],[2,1,0]]...);   #vertici del modello 3D
EV = [[1,2],[2,3],[3,4],[4,1],[1,5]];             #spigoli del modello
bb = [[0.0 1.0; 0.0 0.0], [1.0 1.0; 0.0 1.0], [0.0 1.0; 1.0 1.0], [0.0 0.0; 0.0 1.0], [0.0 2.0; 0.0 1.0]];  #bounding box
dict = OrderedDict([0.0, 1.0] => [1, 3],[1.0, 1.0] => [2],[0.0, 0.0] => [4],[0.0, 2.0] => [5])  #dizionario intervallo/indice
cov = [[4, 1, 3, 5, 2], [1, 3, 5, 2], [4, 1, 3, 5, 2], [4, 1, 3, 5], [4, 1, 3, 5, 2]]    #intersezioni tra bounding box

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## Funzioni di supporto

funzione che crea un albero di supporto per la funzione principale **boxcovering**; nel particolare dato un dizionario ordinato crea un intervalTrees cioè una struttura dati che contiene intervalli e che consente di trovare in modo efficiente tutti gli intervalli che si sovrappongono a un determinato intervallo o punto.

In [19]:
function createIntervalTree(boxdict::AbstractDict{Array{Float64,1},Array{Int64,1}})
    tree = IntervalTrees.IntervalMap{Float64,Array}()
    for (key, boxset) in boxdict
        tree[tuple(key...)] = boxset
    end
    return tree
end

createIntervalTree (generic function with 1 method)

In [20]:
t=createIntervalTree(dict)  #creazione dell'albero t

IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

### addIntersection

addIntersection aggiunge in 'covers' in i-esima posizione tutti i bounding box che intersecano l'i-esimo bounding box


In [21]:
function addIntersection(covers::Array{Array{Int64,1},1}, i::Int64, iterator)
    splice!(covers[i],1)		#splice serve a togliere gli zeri iniziali all'interno di covers
    @threads for x in collect(iterator)
        append!(covers[i],x.value)
    end
end

addIntersection (generic function with 1 method)

## Versione iniziale di boxcovering

boxcovering calcola quali bounding box si intersecano tra loro.

In [22]:
function boxcovering(bboxes, index, tree)
	covers = [[] for k=1:length(bboxes)]
	for (i,boundingbox) in enumerate(bboxes)
		extent = bboxes[i][index,:]
		iterator = IntervalTrees.intersect(tree, tuple(extent...))
		for x in iterator
			append!(covers[i],x.value)
		end
	end
	return covers
end

boxcovering (generic function with 1 method)

In [34]:
@btime boxcovering(bb, 1, t)   #8.996 μs

  8.996 μs (86 allocations: 4.67 KiB)


5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [ ]:
@code_warntype boxcovering(bb, 1, t)

boxcovering è type stable ma la variabile covers è un array di Any. Si procede tipizzando covers e dividendo la funzione in microtask.

### Versione parallelizzata di boxcovering

In [25]:
function boxcoveringMOD(bboxes, index, tree)
    covers = [[zero(eltype(Int64))] for k=1:length(bboxes)]		#zero(eltype(Int64)) serve per rendere covers type stable
    @threads for (i,boundingbox) in collect(enumerate(bboxes))
        extent = bboxes[i][index,:]
        iterator = IntervalTrees.intersect(tree, tuple(extent...))
        addIntersection(covers, i, iterator)
    end
    return covers
end

boxcoveringMOD (generic function with 1 method)

In [26]:
cov = boxcoveringMOD(bb, 1, t)

5-element Vector{Vector{Int64}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

In [27]:
ex = bb[1][1,:]

2-element Vector{Float64}:
 0.0
 1.0

In [28]:
it = IntervalTrees.intersect(t, tuple(ex...))


IntervalTrees.IntervalIntersectionIterator{typeof(IntervalTrees.true_cmp), Float64, IntervalValue{Float64, Array}, 64}(IntervalTrees.true_cmp, IntervalTrees.Intersection{Float64, IntervalValue{Float64, Array}, 64}(0, #undef), IntervalTrees.IntervalBTree{Float64, IntervalValue{Float64, Array}, 64}, Interval{Float64}
(0.0,1.0))


(0.0,0.0) => [4]
(0.0,1.0) => [1, 3]
(0.0,2.0) => [5]
(1.0,1.0) => [2]

In [35]:
@btime boxcoveringMOD(bb, 1, t)    #4.550 μs

  4.550 μs (82 allocations: 3.02 KiB)


5-element Vector{Vector{Any}}:
 [4, 1, 3, 5, 2]
 [1, 3, 5, 2]
 [4, 1, 3, 5, 2]
 [4, 1, 3, 5]
 [4, 1, 3, 5, 2]

## TEST

In [33]:
using Test

@testset "boxcovering Tests" begin
    V,(VV,EV,FV,CV) = Lar.cuboidGrid([2,2,2],true)
    W,_ = Lar.apply(Lar.r(1,1,pi/6),(V,[VV,EV,FV,CV]))
    cellpoints = [ W[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    dict = Lar.coordintervals(1,bboxes)
    @test typeof(dict) == OrderedDict{Array{Float64,1},Array{Int64,1}}
    @test length(Lar.coordintervals(1,bboxes)) == 54
    @test length(Lar.coordintervals(2,bboxes)) == 54
    @test length(Lar.coordintervals(3,bboxes)) == 54

    V,(VV,EV,FV) = Lar.cuboidGrid([2,1],true)
    cellpoints = [ V[:,EV[k]]::Lar.Points for k=1:length(EV) ]
    bboxes = [hcat(Lar.boundingbox(cell)...) for cell in cellpoints]
    @test bboxes == [[0.0 0.0; 0.0 1.0],
    [1.0 1.0; 0.0 1.0],
    [2.0 2.0; 0.0 1.0],
    [0.0 1.0; 0.0 0.0],
    [0.0 1.0; 1.0 1.0],
    [1.0 2.0; 0.0 0.0],
    [1.0 2.0; 1.0 1.0]]
    xboxdict = Dict(
     [0.0, 0.0] => [1],
     [1.0, 1.0] => [2],
     [2.0, 2.0] => [3],
     [0.0, 1.0] => [4, 5],
     [1.0, 2.0] => [6, 7])
    @test xboxdict == Lar.coordintervals(1,bboxes)
    xs = IntervalTrees.IntervalMap{Float64, Array}()
    for (key,boxset) in xboxdict
        xs[tuple(key...)] = boxset
    end
   @test typeof(xs) ==
    IntervalTrees.IntervalBTree{Float64,
    IntervalValue{Float64,Array},64}
end

Test Summary:     | Pass  Total
boxcovering Tests |    7      7


Test.DefaultTestSet("boxcovering Tests", Any[], 7, false, false)

![test di boxcovering](https://github.com/MarcoCap13/LARSplitting2D/blob/main/docs/test/boxcovering_test.png?raw=true)

### Benchmark della funzione iniziale e modificata

funzione iniziale:

In [36]:
@benchmark boxcovering(bb, 1, t)   

BenchmarkTools.Trial: 10000 samples with 3 evaluations.
 Range (min … max):   8.982 μs …  4.647 ms  ┊ GC (min … max): 0.00% … 98.92%
 Time  (median):      9.499 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   12.892 μs ± 64.329 μs  ┊ GC (mean ± σ):  6.90% ±  1.40%

  █▆▂▁     ▁▂▂▂▁                                              ▁
  ████████▇███████▇▇▇▇▆▇▆▇▇▆▆▆▆▅▆▅▅▆▆▆▆▅▅▅▅▅▆▅▅▅▄▅▅▄▅▄▁▁▄▅▄▄▅ █
  8.98 μs      Histogram: log(frequency) by time      52.9 μs <

 Memory estimate: 4.67 KiB, allocs estimate: 86.

funzione modificata:

In [37]:
@benchmark boxcoveringMOD(bb, 1, t)   

BenchmarkTools.Trial: 10000 samples with 7 evaluations.
 Range (min … max):  4.553 μs …  1.526 ms  ┊ GC (min … max): 0.00% … 99.33%
 Time  (median):     4.763 μs              ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.307 μs ± 29.928 μs  ┊ GC (mean ± σ):  8.81% ±  1.98%

  █▆▄▂            ▁▁▂▁                                       ▁
  █████▇▆▆▇▆▆▆▆▆▆▇██████▇▆▆▆▆▇▅▅▆▆▆▅▅▅▅▅▅▄▅▅▄▄▅▃▄▅▃▄▅▅▅▄▅▄▄▃ █
  4.55 μs      Histogram: log(frequency) by time     19.2 μs <

 Memory estimate: 3.02 KiB, allocs estimate: 82.